In [1]:
import os 
os.chdir('/Users/egill/dev/genki/genki-signals')

In [2]:
from genki_signals.buffers import DataBuffer

buf = DataBuffer(max_size=100)

In [7]:
for i in range(1000):
    if i % 2 == 0:
        buf.append({'a': i, 'b': i})
    else:
        buf.append({'a': i})

In [27]:
import numpy as np
from genki_signals.data_sources import SineWave

sine = SineWave(amplitude=5, frequency=3, phase=0)

In [28]:
from genki_signals.data_sources import Sampler

source = Sampler({'sine': sine}, sample_rate=100)

In [29]:
from threading import Thread
import time

buf = DataBuffer(max_size=400)

def run_source(source, buffer):
    source.start()
    while True:
        buffer.extend(source.read())
        time.sleep(1 / 25)
        if not source.is_active:
            return
        
t = Thread(target=run_source, args=(source, buf))
t.start()

In [30]:
buf.plot('sine')

Figure(axes=[Axis(label='t', scale=LinearScale()), Axis(label='sine', orientation='vertical', scale=LinearScal…

In [32]:
from ipywidgets import interact, IntSlider, FloatSlider

@interact(amplitude=FloatSlider(min=1, max=5, step=0.5, value=5), frequency=IntSlider(min=1, max=7, value=3))
def set_wave_params(amplitude, frequency):
    sine.amplitude = amplitude
    sine.frequency = frequency

interactive(children=(FloatSlider(value=5.0, description='amplitude', max=5.0, min=1.0, step=0.5), IntSlider(v…

In [33]:
source.stop()

In [35]:
from genki_signals.data_sources import MicDataSource, WaveDataSource, MouseDataSource, KeyboardDataSource

wave_source = WaveDataSource(ble_address='5C72E785-DAF5-1E32-599D-EBF56B8ECD5B')


In [36]:
buf = DataBuffer(max_size=400)

t = Thread(target=run_source, args=(wave_source, buf))
t.start()

Connecting to wave at address 5C72E785-DAF5-1E32-599D-EBF56B8ECD5B
Connected to Wave


In [39]:
buf

DataBuffer(max_size=400, data=gyro: (3, 400)
acc: (3, 400)
mag: (3, 400)
raw_pose: (4, 400)
current_pose: (4, 400)
euler: (3, 400)
linacc: (3, 400)
peak: (400,)
peak_norm_velocity: (400,)
timestamp_us: (400,)
grav: (3, 400)
acc_glob: (3, 400)
linacc_glob: (3, 400))

In [40]:
buf.plot('gyro')

Figure(axes=[Axis(label='t', scale=LinearScale()), Axis(label='gyro', orientation='vertical', scale=LinearScal…

In [41]:
wave_source.stop()

Got a cancel message, exiting.


In [42]:
from genki_signals.data_sources import RandomNoise

noise_source = RandomNoise()

source = Sampler({
    'sine_1': SineWave(amplitude=5, frequency=3, phase=0),
    'sine_2': SineWave(amplitude=3, frequency=7, phase=0.2),
    'noise': noise_source
}, sample_rate=100)

In [43]:
import genki_signals.signals as s

derived_signals = [
    s.Sum('sine_1', 'sine_2', name='composite_sine'),
    s.Sum('composite_sine', 'noise', name='wave_with_noise'),
    s.FourierTransform('wave_with_noise', name='wave_spectrum', window_size=256, window_overlap=64)
]

In [44]:
from genki_signals.system import System

sys_1 = System(source, derived_signals)

In [45]:
buf = DataBuffer(max_size=400)

t = Thread(target=run_source, args=(sys_1, buf))
t.start()

In [46]:
buf

DataBuffer(max_size=400, data=timestamp: (349,)
sine_1: (349,)
sine_2: (349,)
noise: (349,)
composite_sine: (349,)
wave_with_noise: (349,)
wave_spectrum: (129, 349))

In [47]:
buf.plot('wave_with_noise')

Figure(axes=[Axis(label='t', scale=LinearScale()), Axis(label='wave_with_noise', orientation='vertical', scale…

In [48]:
from ipywidgets import interact, FloatSlider

@interact(noise=FloatSlider(min=0.1, max=3.0, step=0.1, value=1))
def set_noise(noise):
    noise_source.amplitude = noise

interactive(children=(FloatSlider(value=1.0, description='noise', max=3.0, min=0.1), Output()), _dom_classes=(…

In [53]:
buf.plot('wave_spectrum', plot_type='spectrogram', window_size=256, sample_rate=100)

Figure(axes=[Axis(label='Hz', scale=LinearScale()), Axis(label='db', orientation='vertical', scale=LinearScale…

In [58]:
sys_1.stop()

In [55]:
mic_source = MicDataSource()

sys_2 = System(mic_source, [])

In [59]:
buf = DataBuffer(max_size=400)

t = Thread(target=run_source, args=(sys_2, buf))
t.start()

In [61]:
sys_2.add_derived_signal(s.FourierTransform('audio', name='audio_spectrum', window_size=1024, window_overlap=0))

In [64]:
buf.plot('audio_spectrum', plot_type='spectrogram', window_size=1024, sample_rate=mic_source.sample_rate)

Figure(axes=[Axis(label='Hz', scale=LinearScale()), Axis(label='db', orientation='vertical', scale=LinearScale…

In [65]:
sys_2.stop()

In [66]:
mouse_source = Sampler({'mouse_position': MouseDataSource()}, sample_rate=100)

buf = DataBuffer(max_size=400)

sys_3 = System(mouse_source, derived_signals=[])

t = Thread(target=run_source, args=(sys_3, buf))
t.start()

In [68]:
buf.plot('mouse_position', plot_type='trace2D')

Figure(axes=[Axis(label='x', scale=LinearScale()), Axis(label='y', orientation='vertical', scale=LinearScale()…

In [69]:
sys_3.add_derived_signal(s.Differentiate('mouse_position', name='mouse_velocity'))

In [70]:
from genki_signals.models.letter_detection_model import SimpleGruModel

example_rnn_model = SimpleGruModel.load_from_checkpoint("genki_signals/models/stc_detector_final-epoch=15-val_loss=0.53.ckpt")

In [71]:
sys_3.add_derived_signal(s.Inference(example_rnn_model, input_signals=['mouse_velocity'], 
                                     stateful=True, name='model_inference'))

In [72]:
buf

DataBuffer(max_size=400, data=timestamp: (400,)
mouse_position: (2, 400)
mouse_velocity: (2, 400)
model_inference: (4, 231))

In [75]:
class_names=['background', 'square', 'triangle', 'circle']

buf.plot('model_inference', plot_type='histogram', class_names=class_names)

Figure(axes=[Axis(scale=OrdinalScale()), Axis(label='Probability', orientation='vertical', scale=LinearScale()…

In [76]:
sys_3.stop()